# survey

> This module handles observation targets, survey duration, and other similar functionality. Currently only for MW surveys.

In [1]:
#| default_exp survey

In [1]:
#| hide
from nbdev.showdoc import *
import numpy as np
from LensCalcPy.pbh import Pbh

In [2]:
#| export
class Survey:
    """
    A class to represent a survey.
    """

    def __init__(self, 
                 l:float, # Galactic longitude in degrees
                 b: float, # Galactic latitude in degrees
                 source_dist:float, # Distance to observation source in kpc
                 obs_time: float # Observation time in hours
                 ):
     
        self.l = l 
        self.b = b 
        self.source_dist = source_dist 
        self.earth_dist = 8.5 
        self.obs_time = obs_time 
        self.pbh = None # PBH population
        self.ffp = None # FFP population

    def __str__(self) -> str:
        return f"Survey(l={self.l}, b={self.b}, source_dist={self.source_dist})"
    __repr__ = __str__
    
    def dist_mw(self, d):
        """returns the distance to the Milky Way center in kpc of a point a distance d to the Sun in kpc"""
        return np.sqrt(d**2 + self.earth_dist**2 - 2*d*self.earth_dist*np.cos(np.radians(self.l))*np.cos(np.radians(self.b)))
    
    def add_pbh(self, 
                m_pbh: float = 1, # mass of each PBH in solar masses
                f_dm: float = 1 # fraction of DM in PBHs
                ):
        """adds a PBH population to the survey"""
        self.pbh = Pbh(m_pbh, f_dm)
        return
    


In [3]:
s = Survey(1.0, -1.03, 9.0, 7.0)
s

Survey(l=1.0, b=-1.03, source_dist=9.0)

In [4]:
s.dist_mw(5)

3.5038090867517475

In [5]:
s.add_pbh(1, 1)
print(s.pbh.density(5))

22557800.0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()